## Use Case: Automated Financial Report Generation and Review

*[Coding along with the Udemy online course [Mastering AutoGen: Building Multi-Agent Systems](https://www.udemy.com/course/autogen-agent-systems/?couponCode=MTST7102224A2) by Paulo Dichone]*

__Scenario:__ A financial services firm that wants to automate the process of producing monthly financial reports.

__Workflow:__

1. __Data Collection Agent__ collects and aggregates finacial data from various sources,
2. then passes collected information to __Report Genaration Agent__ who's responsible for producing details for the financial report based on the aggregated data.
3. Next level __Report Review__:
   1. __Accuracy Review Agent__ checks the report for accuracy and consistency.
   2. __Compliance Review Agent__ assures the report complies with financial regulations and company standards.
4. Next level __Executive Summary__:
   1. __Summary Generation Agent__ summarizes report for executive representations.
   2. __Feedback Agent__ collects feedback from (on?) the created summarys.


In [1]:
from autogen import AssistantAgent, UserProxyAgent
import pandas as pd
from pprint import pprint

In [2]:
api_key = pd.read_csv("~/tmp/chat_gpt/agentic-design-1.txt", sep=" ", header=None)[0][0]
print("Don't be a fool and send your api key to GitHub!")

Don't be a fool and send your api key to GitHub!


In [3]:
llm_config = {
    "model": "gpt-4o-mini",
    "temperature": 0.4,
    "api_key": api_key
    }
print("Don't be a fool and send your api key to GitHub!")

Don't be a fool and send your api key to GitHub!


In [4]:
# define the data aggregation agent
data_aggregation_agent = AssistantAgent(
    name="Data_Aggregation_Agent",
    llm_config=llm_config,
    system_message="""
        You collect and aggregate financial data from the provided CSV file for the monthly financial report.

    """,
)

In [5]:
# define the report generation agent
report_generation_agent = AssistantAgent(
    name="Report_Generation_Agent",
    llm_config=llm_config,
    system_message="""
    You generate detailed financial reports based on the aggregated data.
    """,
)

In [6]:
# define the accuracy review agent
accuracy_review_agent = AssistantAgent(
    name="Accuracy_Review_Agent",
    llm_config=llm_config,
    system_message="""
    You check the financial report for accuracy and consistency.
    """,
)

In [7]:
# define the compliance review agent
compliance_review_agent = AssistantAgent(
    name="Compliance_Review_Agent",
    llm_config=llm_config,
    system_message="""
    You ensure the financial report complies with financial regulations and standards.
    """,
)

In [8]:
# define the summary generation agent
summary_generation_agent = AssistantAgent(
    name="Summary_Generation_Agent",
    llm_config=llm_config,
    system_message="""
    You summarize the financial report for executive presentations.
    """,
)

In [9]:
# define the feedback agent
feedback_agent = AssistantAgent(
    name="Feedback_Agent",
    llm_config=llm_config,
    system_message="""
    You collect feedback from executives on the summary of the financial report.
    """,
)

In [10]:
# define the user proxy agent
user_proxy = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "my_code",
        "use_docker": False,
    },
)

In [21]:
# function to read CSV file and prepare data for aggregation
def read_csv_file():
    print("Reading CSV file...")
    df = pd.read_csv("../assets/data/financial_data.csv")
    return df.to_dict()

In [16]:
# register nested chats with the user proxy agent
user_proxy.register_nested_chats(
    [
        {
            "recipient": report_generation_agent,
            "message": lambda recipient, messages, sender, config: f"Generate a detailed financial report based on the following data: {read_csv_file()}",
            "summary_method": "last_msg",
            "max_turns": 1,
        },
        {
            "recipient": accuracy_review_agent,
            "message": lambda recipient, messages, sender, config: f"Check this report for accuracy and consistency: {messages[-1]['content']}",
            "summary_method": "last_msg",
            "max_turns": 1,
        },
        {
            "recipient": compliance_review_agent,
            "message": lambda recipient, messages, sender, config: f"Ensure this report complies with financial regulations and standards: {messages[-1]['content']}",
            "summary_method": "last_msg",
            "max_turns": 1,
        },
        {
            "recipient": summary_generation_agent,
            "message": lambda recipient, messages, sender, config: f"Summarize the financial report for an executive presentation: {messages[-1]['content']}",
            "summary_method": "last_msg",
            "max_turns": 1,
        },
        {
            "recipient": feedback_agent,
            "message": lambda recipient, messages, sender, config: f"Collect feedback on this summary from executives: {messages[-1]['content']}",
            "summary_method": "last_msg",
            "max_turns": 1,
        },
    ],
    trigger=data_aggregation_agent,
)

In [17]:
# define the initial data aggregation task
initial_task = (
    """Collect and aggregate financial data for the monthly financial report."""
)

In [22]:
# start the nested chat
user_proxy.initiate_chat(
    recipient=data_aggregation_agent,
    message=initial_task,
    max_turns=2,
    summary_method="last_msg",
)

User (to Data_Aggregation_Agent):

Collect and aggregate financial data for the monthly financial report.

--------------------------------------------------------------------------------
Data_Aggregation_Agent (to User):

To assist you in collecting and aggregating financial data for your monthly financial report, I would need you to provide the CSV file containing the relevant financial data. Once you upload the file, I can help you analyze and summarize the information.

--------------------------------------------------------------------------------
Reading CSV file...

********************************************************************************
Starting a new chat....

********************************************************************************
User (to Report_Generation_Agent):

Generate a detailed financial report based on the following data: {'Date': {0: '2023-01-01', 1: '2023-02-01', 2: '2023-03-01'}, 'Revenue': {0: 100000, 1: 120000, 2: 130000}, 'Expenses': {0: 70000,

ChatResult(chat_id=None, chat_history=[{'content': 'Collect and aggregate financial data for the monthly financial report.', 'role': 'assistant', 'name': 'User'}, {'content': 'To assist you in collecting and aggregating financial data for your monthly financial report, I would need you to provide the CSV file containing the relevant financial data. Once you upload the file, I can help you analyze and summarize the information.', 'role': 'user', 'name': 'Data_Aggregation_Agent'}, {'content': '### Feedback Summary on Financial Report\n\n**Overall Impression:**\nThe financial report is comprehensive and well-structured, effectively summarizing the company\'s financial performance for Q1 2023. The clarity of the data presentation and the logical flow of the analysis make it easy for stakeholders to understand the key financial metrics.\n\n**Strengths:**\n1. **Clarity and Structure:** The report is organized into clear sections, making it easy to navigate and understand.\n2. **Key Metrics H